In [1]:
import os
import shutil
import logging
import pandas as pd
import numpy as np
from fastdtw import fastdtw
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# Clustering

In [ ]:
def load_all_stock_series(directory):
    stock_series = {}
    stock_file_paths = {}
    try:
        for filename in os.listdir(directory):
            if filename.endswith('.parquet'):
                symbol = filename.split('.')[0]
                file_path = os.path.join(directory, filename)
                stock_series[symbol] = pd.read_parquet(file_path)
                stock_file_paths[symbol] = file_path
                logging.info(f"✅ File {filename} successfully loaded.")
        return stock_series, stock_file_paths
    except FileNotFoundError as e:
        logging.error(f"❌ Directory {directory} not found: {e}.")
        return {}, {}
    

def compute_dtw_distance_matrix(stock_series, column='Close'):
    symbols = list(stock_series.keys())
    n = len(symbols)
    dtw_matrix = np.zeros((n, n))

    for i in range(n):
        series_i = np.asarray(stock_series[symbols[i]][column]).squeeze().flatten()
        if series_i.ndim != 1:
            logging.error(f"❌ Series for {symbols[i]} is not 1-D after flatten, shape: {series_i.shape}")
            raise ValueError(f"Series for {symbols[i]} is not 1-D after flatten, shape: {series_i.shape}")

        for j in range(i + 1, n):
            series_j = np.asarray(stock_series[symbols[j]][column]).squeeze().flatten()
            if series_j.ndim != 1:
                logging.error(f"❌ Series for {symbols[j]} is not 1-D after flatten, shape: {series_j.shape}")
                raise ValueError(f"Series for {symbols[j]} is not 1-D after flatten, shape: {series_j.shape}")

            distance, _ = fastdtw(series_i, series_j, dist=lambda x, y: abs(x - y))
            dtw_matrix[i, j] = distance
            dtw_matrix[j, i] = distance

    logging.info("✅ DTW distance matrix computation complete.")
    return dtw_matrix, symbols


def hierarchical_clustering_dtw(dtw_matrix, symbols, n_clusters=5):
    """
    Perform hierarchical clustering on the DTW distance matrix.

    Parameters:
        dtw_matrix (np.array): DTW distance matrix.
        symbols (list): List of stock symbols corresponding to the matrix indices.
        n_clusters (int): Number of clusters to form.

    Returns:
        dict: Mapping of stock symbol to assigned cluster label.
    """
    condensed_matrix = squareform(dtw_matrix)
    linked = linkage(condensed_matrix, method='ward')
    cluster_labels = fcluster(linked, n_clusters, criterion='maxclust')

    clusters = {symbol: cluster for symbol, cluster in zip(symbols, cluster_labels)}
    logging.info("Hierarchical clustering complete.")
    return clusters


def save_stocks_to_cluster_dirs(clusters, data_directory, output_directory):
    try:
        cluster_dirs = {cluster: os.path.join(output_directory, f"cluster_{cluster}")
                        for cluster in set(clusters.values())}
        for path in cluster_dirs.values():
            os.makedirs(path, exist_ok=True)

        for stock, cluster in clusters.items():
            source_file = os.path.join(data_directory, f"{stock}.parquet")
            dest_file = os.path.join(cluster_dirs[cluster], f"{stock}.parquet")
            if os.path.exists(source_file):
                shutil.copy(source_file, dest_file)
                logging.info(f"Copied {stock}.parquet to cluster_{cluster} directory.")
            else:
                logging.warning(f"❌ File {source_file} not found!")
    except FileNotFoundError as e:
        logging.error(f"❌ Directory {data_directory} not found: {e}")


if __name__ == "__main__":
    data_directory = r"/Users/akramchakrouni/Projects/time-series-forecasting-cluserting/data/chronos"
    output_directory = r"/Users/akramchakrouni/Projects/time-series-forecasting-cluserting/clusters/dtw"
    n_clusters = 5

    stock_series, stock_file_paths = load_all_stock_series(data_directory)
    dtw_matrix, symbols = compute_dtw_distance_matrix(stock_series, column='Close')
    clusters = hierarchical_clustering_dtw(dtw_matrix, symbols, n_clusters=n_clusters)

    save_stocks_to_cluster_dirs(clusters, data_directory, output_directory)

# Finding ideal amount of clusters

In [7]:
def find_best_num_clusters(distance_matrix, method='ward', min_k=2, max_k=10):
    """
    Try different numbers of clusters (k) for hierarchical clustering
    and compute the silhouette score for each. Return the best k.

    Parameters:
        distance_matrix (np.ndarray): Your precomputed DTW distance matrix (NxN).
        method (str): Linkage method to use ('ward', 'complete', etc.).
        min_k (int): Minimum number of clusters to try.
        max_k (int): Maximum number of clusters to try.

    Returns:
        best_k (int): The number of clusters that yields the best silhouette score.
        best_score (float): The best silhouette score found.
        scores_dict (dict): A dictionary of k -> silhouette score.
    """
    # Condense the NxN matrix into a 1D array for scipy’s hierarchical clustering
    condensed = squareform(distance_matrix)
    
    # Perform hierarchical clustering once outside the loop
    # (linkage can be reused for different cluster cuts)
    Z = linkage(condensed, method=method)

    best_k = min_k
    best_score = -1
    scores_dict = {}

    for k in range(min_k, max_k + 1):
        # Cut the dendrogram to form k clusters
        cluster_labels = fcluster(Z, k, criterion='maxclust')

        # Compute silhouette score on the distance matrix
        # metric='precomputed' means we pass the NxN distance matrix directly
        score = silhouette_score(distance_matrix, cluster_labels, metric='precomputed')

        scores_dict[k] = score
        if score > best_score:
            best_score = score
            best_k = k

    return best_k, best_score, scores_dict

best_k, best_score, all_scores = find_best_num_clusters(dtw_matrix, method='ward', min_k=2, max_k=10)
print("Best number of clusters:", best_k)
print("Best silhouette score:", best_score)
print("Scores for each k:", all_scores)

Best number of clusters: 2
Best silhouette score: 0.6212347846302031
Scores for each k: {2: 0.6212347846302031, 3: 0.35761369428187617, 4: 0.37768199606192765, 5: 0.32130582084910864, 6: 0.3248682867115323, 7: 0.30652267040939823, 8: 0.3172446176425021, 9: 0.24567084205401182, 10: 0.23804313443212055}


In [10]:
def find_best_num_clusters_kneedle(distance_matrix, method='ward'):
    """
    Determine the optimal number of clusters using the elbow method and the kneedle algorithm.
    
    Parameters:
        distance_matrix (np.ndarray): Precomputed DTW distance matrix (NxN).
        method (str): Linkage method to use (e.g., 'ward', 'complete', etc.).
    
    Returns:
        best_k (int): The number of clusters suggested by the elbow.
        Z (np.ndarray): The linkage matrix used for clustering.
    """
    # Convert the full (NxN) distance matrix to condensed form.
    condensed = squareform(distance_matrix)
    # Compute the linkage matrix.
    Z = linkage(condensed, method=method)
    n = distance_matrix.shape[0]
    
    # In hierarchical clustering, each row in Z corresponds to a merge.
    # When there are n points, Z has (n-1) rows.
    # The number of clusters after each merge goes from n down to 1.
    # We extract the merge distances (Z[:, 2]) and reverse them so they correspond 
    # to cluster counts from n down to 2.
    cluster_counts = np.arange(n, 1, -1)
    rev_distances = Z[:, 2][::-1]
    
    # Use the KneeLocator to detect the "knee" in the plot of cluster count vs. merge distance.
    # The knee point represents a significant jump in the merging distance.
    kneedle = KneeLocator(cluster_counts, rev_distances, curve='convex', direction='increasing')
    best_k = kneedle.knee if kneedle.knee is not None else n // 2  # Fallback if no knee detected

    logging.info(f"Optimal number of clusters suggested by the elbow: {best_k}")
    return best_k, Z

# Example usage:
best_k, Z = find_best_num_clusters_kneedle(dtw_matrix, method='ward')
print("Optimal number of clusters:", best_k)


Optimal number of clusters: 45
